In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from deepar.time_series import TimeSeriesTrain, TimeSeriesTest
from deepar.learner import DeepARLearner
import tensorflow as tf
import numpy as np
import os 
import category_encoders
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        exit(-1)


2022-11-13 16:01:45.020903: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/root/miniconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
train = pd.read_csv("data/train.csv", parse_dates = ["datetime"])
test = pd.read_csv("data/test.csv", parse_dates = ["datetime"])

In [5]:
train['datetime'] = pd.date_range(start='2011-01-01', periods=len(train), freq='D')
train['series'] = 'Unique'

In [6]:
train_df = train[:10880]
val = train[10880:]

In [7]:
ds = TimeSeriesTrain(df = train_df, date_col='datetime', target_col='count', groupby_col ='series', freq='D', feat_dynamic_reals=['temp', 'humidity'], feat_static_cats=[ 'series'], feat_static_reals=['season', 'holiday', 'workingday', 'weather',])

/root/autodl-tmp/mydeepar/deepar/time_series.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.groupby_col] = data[self.groupby_col].astype(str)
/root/autodl-tmp/mydeepar/deepar/time_series.py:190: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_names = self.df[cat].append(pd.Series(['dummy_test_category']))


In [8]:
DeepARLearner(ds).model.summary()

number of cats : 5, number of conts: 8


2022-11-13 16:02:34.365368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 16:02:35.164322: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14255 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:52:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(64, 20)]           0           []                               
                                                                                                  
 input_2 (InputLayer)           [(64, 20)]           0           []                               
                                                                                                  
 input_3 (InputLayer)           [(64, 20)]           0           []                               
                                                                                                  
 input_4 (InputLayer)           [(64, 20)]           0           []                               
                                                                                              

In [9]:
learner = DeepARLearner(ds, cell_type='lstm')

number of cats : 5, number of conts: 8


In [10]:
learner.fit(1)

2022-11-13 16:02:47.140302: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(<tf.Tensor: shape=(), dtype=float32, numpy=0.10239862>, 1)

In [12]:
val_ds = TimeSeriesTest(ds, val.drop(columns=['count']))

In [13]:
pred = learner.predict(val_ds, samples=1, return_in_sample_predictions=False)

/root/miniconda3/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


horizon idx :0
in-sample ancestral sampling
horizon idx :1
learn from test samples
horizon idx :2
learn from test samples
horizon idx :3
learn from test samples
horizon idx :4
learn from test samples
horizon idx :5
learn from test samples


In [14]:
pred

array([[[ 30],
        [ 18],
        [120],
        [ 14],
        [ 17],
        [ 47]]])